In [1]:
!pip install portalocker
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import argparse
import logging
import time

import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchtext.vocab import GloVe
from tqdm import tqdm

torch.autograd.set_detect_anomaly(True)

FILL = '_FILL_'

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

### Constants

In [3]:
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cpu"
EMBED_DIM = 300
LR = 1.0
BATCH_SIZE = 128
NUM_EPOCHS = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

### Get the tokenizer
- Use the WordLevel tokenizer.


In [4]:
# Get basic tokenizer
basic_english_tokenizer =  get_tokenizer("basic_english")

In [5]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [7]:
# Needed later; set this to the tokenizer above
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary

In [8]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield TOKENIZER(text)

In [9]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<pad>', '<unk>'))

# Make the default index the same as that of the unk_token.
VOCAB.set_default_index(VOCAB['<unk>'])

### Get GLOVE embeddings

In [ ]:
# Get the GloVe embeddings; this will be slow
GLOVE = GloVe()

.vector_cache/glove.840B.300d.zip: 2.18GB [06:49, 5.31MB/s]                            
 88%|████████▊ | 1928269/2196017 [05:23<00:41, 6444.68it/s]

In [12]:
len(GLOVE), GLOVE.vectors.shape


(2196017, torch.Size([2196017, 300]))

### Helper functions

In [13]:
def text_pipeline(text):
    return VOCAB(TOKENIZER(text))

def label_pipeline(label):
    return int(label) - 1

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [14]:
# Loop through the batch and transform into tensors
def collate_batch(batch):
    label_list, text_list = [], []
    for _label, _text in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        label_list.append(label_pipeline(_label))
        # Return a list of ints.
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        # Append to text_list
        text_list.append(processed_text.clone().detach())
    # Pad and make into tensors as needed
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)
    return label_list.to(DEVICE), text_list.to(DEVICE)


### Get the data

In [15]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
num_class = len(set([label for (label, _) in train_iter]))
# What are the classes?
print(f"The number of classes is {num_class} ...")

The number of classes is 4 ...


### Set up the model

Good reference on this type of model
- CNN for Sentence Classification: https://arxiv.org/pdf/1408.5882.pdf

You can build CNN models with either CNN1d or CNN2d.

In [16]:
class CNN1dTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, num_class, embed_dim=300, use_pretrained=True, fine_tune_embeddings=True, debug=False):
        super(CNN1dTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PADDING_IDX)

        if use_pretrained:
            self.embedding.weight.requires_grad = False
            for i in range(vocab_size):
                token = VOCAB.lookup_token(i)
                if token in GLOVE.stoi:
                    self.embedding.weight[i,:] = GLOVE.get_vecs_by_tokens(TOKENIZER(token), lower_case_backup=True)
            self.embedding.weight.requires_grad = True
        else:
            self.init_weights()

        if not fine_tune_embeddings:
            self.embedding.weight.requires_grad = False

        self.cnn1 = nn.Conv1d(embed_dim, 1, 1)
        self.cnn2 = nn.Conv1d(embed_dim, 1, 2)
        self.cnn3 = nn.Conv1d(embed_dim, 1, 3)
        self.cnn4 = nn.Conv1d(embed_dim, 1, 4)

        self.fc = nn.Linear(3, num_class)
        self.debug = debug

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)

        if self.debug:
            print('embedding', embedded.shape)

        embedded = embedded.transpose(1, 2)

        cnn1 = self.cnn1(embedded)

        if self.debug:
            print('cnn1', cnn1.shape)

        cnn1 = nn.MaxPool1d(cnn1.size(-1))(cnn1).squeeze(-1)

        if self.debug:
            print('cnn1 after max pool', cnn1.shape)

        cnn2 = nn.ReLU()(self.cnn2(embedded))

        if self.debug:
            print('cnn2', cnn2.shape)

        cnn3 = nn.ReLU()(self.cnn3(embedded))

        if self.debug:
            print('cnn3', cnn3.shape)

        cnn4 = nn.ReLU()(self.cnn4(embedded))

        if self.debug:
            print('cnn4', cnn4.shape)

        cnn2 = nn.MaxPool1d(cnn2.size(-1))(cnn2).squeeze(-1)
        cnn3 = nn.MaxPool1d(cnn3.size(-1))(cnn3).squeeze(-1)
        cnn4 = nn.MaxPool1d(cnn4.size(-1))(cnn4).squeeze(-1)

        if self.debug:
            print('cnn2 after max', cnn2.shape)

        cnn2 = (cnn2+cnn1) / 2.0
        cnn3 = (cnn3+cnn1) / 2.0
        cnn4 = (cnn4+cnn1) / 2.0

        if self.debug:
            print('cnn2 after skip connection', cnn2.shape)

        cnn_concat = torch.cat((cnn2, cnn3, cnn4), -1)

        if self.debug:
            print('cnn concat', cnn_concat.shape)

        self.debug = False

        out = self.fc(cnn_concat)
        return out


### Set up the model

In [17]:
# If this is True, we will initialize the Embedding layer with GLOVE
USE_PRETRAINED = True
# If this is True, we will allow for gradient updates on the nn.Embedding layer
FINE_TUNE_EMBEDDINGS = True

# Set the loss appropriately
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)

model = CNN1dTextClassificationModel(len(VOCAB), num_class, EMBED_DIM, fine_tune_embeddings=FINE_TUNE_EMBEDDINGS,
                                      use_pretrained=USE_PRETRAINED, debug=True).to(DEVICE)

# Set the optimizer to SGD
# Add an L2 regularizer of 0.00001
optimizer = torch.optim.SGD(model.parameters(), lr=LR, weight_decay=0.00001)

# Set the scheduler to StepLR with gamma=0.1 and step_size=1.0
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma=0.1)


### Set up the data

In [19]:
train_iter, test_iter = DATASETS[DATASET]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train, split_valid = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)


### Train the model

In [20]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss, total_batches = 0.0, 0
    total_zero_gradients_percentage = []
    log_interval = 10

    for idx, (label, text) in enumerate(tqdm(dataloader)):
        # Set gradients to zero
        optimizer.zero_grad()

        # Get the predictions
        predicted_label = model(text)

        # Get the loss
        loss = criterion(input=predicted_label, target=label)

        # Do back propagation and get the gradients
        loss.backward()

        with torch.no_grad():
            total_loss += loss.item()
            total_batches += 1

            # Loop through all the parameters in the model and print out the number of parameters that are zero
            # Specifically, for this batch, get the percentage of zero gradients across all parameters
            # Append this to the list above
            total_nonzero_gradients = 0.0
            total_param_count = 0.0

            for name, p in model.named_parameters():
                if p.requires_grad:
                    total_nonzero_gradients += torch.count_nonzero(p.grad.data).item()
                    total_param_count += p.grad.data.numel()

            total_zero_gradients_percentage.append(100.0 - (total_nonzero_gradients / total_param_count) * 100.0)


In [21]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(input=predicted_label, target=label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()

    return total_acc, total_count, total_loss


In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    accu_val, loss_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val,
            loss_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test, loss_test = evaluate(test_dataloader, model, criterion)
print("test accuracy {:8.3f}".format(accu_test))
print("test loss {:8.3f}".format(loss_test))

  3%|▎         | 24/891 [00:13<08:03,  1.79it/s]